# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import numpy as np
import sqlite3
import pandas as pd
import seaborn as nsn
import re
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_colwidth',True)


In [2]:
# load data from database
conn = sqlite3.connect('DisasterResponse.db')
table_name = "disaster_messages"
df = pd.read_sql(f"SELECT * from {table_name}",conn)


conn.close()
df.head(1)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# defining target and features 
X = df['message']
y = df.drop(columns=['id','message','genre'])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Tokenizes and cleans text by splitting into words, lemmatizing, normalizing case, 
    and removing non-alphanumeric tokens.

    Args:
        text (str): The text string to tokenize and clean.

    Returns:
        list of str: A list of processed, lowercase, lemmatized tokens with punctuation removed.
    """ 
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # initiate lemmatizer
    clean_tokens = [WordNetLemmatizer().lemmatize(token).lower().strip() for token in tokens if token.isalnum()]
    return clean_tokens





In [6]:
X

0        Weather update - a cold front from Cuba that could pass over Haiti                                                                                                                                               
1        Is the Hurricane over or is it not over                                                                                                                                                                          
2        Looking for someone but no name                                                                                                                                                                                  
3        UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.                                                                                                             
4        says: west side of Haiti, rest of the country today and tonight                                                    

In [7]:
X.apply(tokenize)

0        [weather, update, a, cold, front, from, cuba, that, could, pas, over, haiti]                                                                                                                                                  
1        [is, the, hurricane, over, or, is, it, not, over]                                                                                                                                                                             
2        [looking, for, someone, but, no, name]                                                                                                                                                                                        
3        [un, report, leogane, destroyed, only, hospital, croix, functioning, needs, supply, desperately]                                                                                                                              
4        [say, west, side, of, haiti, rest, of, the, country, today, and

### 3. Building a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),  
        ('tfidf', TfidfTransformer()),                 
        ('clf', MultiOutputClassifier(RandomForestClassifier()))  
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)


In [10]:
model = pipeline.fit(X_train,y_train)

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:

y_pred = model.predict(X_test)

for i, column in enumerate(y_test.columns):
    print(f"Category: {column}")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Category: related
              precision    recall  f1-score   support

           0       0.78      0.29      0.42      1568
           1       0.81      0.97      0.88      4930
           2       0.58      0.62      0.60        47

    accuracy                           0.81      6545
   macro avg       0.72      0.63      0.63      6545
weighted avg       0.80      0.81      0.77      6545

Category: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5411
           1       0.90      0.41      0.56      1134

    accuracy                           0.89      6545
   macro avg       0.90      0.70      0.75      6545
weighted avg       0.89      0.89      0.87      6545

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6513
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6545
   macro avg       0.5

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6360
           1       0.80      0.02      0.04       185

    accuracy                           0.97      6545
   macro avg       0.89      0.51      0.51      6545
weighted avg       0.97      0.97      0.96      6545

Category: security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6412
           1       0.00      0.00      0.00       133

    accuracy                           0.98      6545
   macro avg       0.49      0.50      0.49      6545
weighted avg       0.96      0.98      0.97      6545

Category: military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6311
           1       0.80      0.03      0.07       234

    accuracy                           0.97      6545
   macro avg       0.88      0.52      0.52      6545
weighted avg       0.96      0.97   

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6247
           1       0.77      0.12      0.21       298

    accuracy                           0.96      6545
   macro avg       0.87      0.56      0.60      6545
weighted avg       0.95      0.96      0.94      6545

Category: other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5699
           1       0.71      0.02      0.03       846

    accuracy                           0.87      6545
   macro avg       0.79      0.51      0.48      6545
weighted avg       0.85      0.87      0.82      6545

Category: infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6077
           1       0.00      0.00      0.00       468

    accuracy                           0.93      6545
   macro avg       0.46      0.50      0.48      6545
weighted avg       0.

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6044
           1       0.92      0.36      0.52       501

    accuracy                           0.95      6545
   macro avg       0.94      0.68      0.75      6545
weighted avg       0.95      0.95      0.94      6545

Category: storm
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5942
           1       0.86      0.45      0.59       603

    accuracy                           0.94      6545
   macro avg       0.90      0.72      0.78      6545
weighted avg       0.94      0.94      0.93      6545

Category: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6476
           1       0.00      0.00      0.00        69

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98

### 6. Improving the model with GridSearch
GridSearch is a technique used in machine learning for tuning the hyperparameters of a model. Hyperparameters are settings on models that need to be specified before training data is used. Unlike model parameters, hyperparameters are not directly learned from the training process. The parameters listed below will evaluate all combinations of these settings to find the optimal configuration for the model. 

Here is a break-down explanation of the parameters:


`vect__max_df`:

This parameter is part of the CountVectorizer step. It controls the maximum document frequency for the terms to be included in the vocabulary.
* Values: [0.75, 1.0]
* 0.75: Terms that appear in more than 75% of the documents will be ignored. This helps remove very common words that might not be    informative.
* 1.0: No terms are ignored based on document frequency, meaning all terms are considered.

`tfidf__use_idf`:

This parameter is part of the TfidfTransformer step. It determines whether to use the inverse document frequency (IDF) component in the TF-IDF calculation.
* Values: [True, False]
* True: The IDF component is used, which helps downscale the importance of terms that appear frequently across documents.
* False: The IDF component is not used, meaning the transformation is based solely on term frequency.

`clf__estimator__n_estimators`:

Purpose: This parameter is part of the RandomForestClassifier within the MultiOutputClassifier. It specifies the number of trees in the forest.
* Values: [50, 100]
* 50: The random forest will consist of 50 trees.
* 100: The random forest will consist of 100 trees. More trees can improve performance but also increase computation time.

`clf__estimator__max_depth`:

Purpose: This parameter is also part of the RandomForestClassifier. It sets the maximum depth of each tree in the forest.
* Values: [None, 10, 20]
* None: The nodes are expanded until all leaves are pure or contain fewer than the minimum number of samples required to split.
* 10: The maximum depth of the trees is limited to 10 levels. This can help prevent overfitting.
* 20: The maximum depth is limited to 20 levels, allowing for more complex trees than a depth of 10.

In [16]:
parameters = {
    'vect__max_df': [0.75, 1.0],
    'tfidf__use_idf': [True, False],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__max_depth': [None, 10, 20]
}
# Parallel Processing: Set n_jobs=-1 in RandomForestClassifier to utilize all 
# available CPU cores for parallel processing
grid_search = GridSearchCV(pipeline, parameters, cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits


/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used sin

[CV] END clf__estimator__max_depth=None, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75; total time= 5.0min


/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__estimator__max_depth=None, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75; total time= 5.2min


...

[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__max_df=1.0; total time= 1.6min


/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__max_df=1.0; total time= 1.6min
[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_df=0.75; total time= 1.5min
[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_df=0.75; total time= 1.4min
[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_df=0.75; total time= 1.1min
[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_df=1.0; total time= 1.0min
[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_df=1.0; total time=  55.2s
[CV] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_df=1.0; total time=  51.1s


/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fd6a3f871c0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_depth': [None, 10, 20],
                         'clf__estimator__n_estimators': [50, 100],
                         'tfidf__use_idf': [True, False],
                         'vect__max_df': [0.75, 1.0]},
             verbose=2)

In [23]:
print(grid_search.best_params_)

{'clf__estimator__max_depth': None, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': True, 'vect__max_df': 0.75}


### 7. Testing  the GridSearch model
Showing the accuracy, precision, and recall of the tuned model with `classification_report`.



In [25]:
y_pred = grid_search.predict(X_test)

# Iterate through each category and print classification report
for i, column in enumerate(y_test.columns):
    print(f"Category: {column}")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Category: related
              precision    recall  f1-score   support

           0       0.78      0.30      0.43      1568
           1       0.81      0.97      0.88      4930
           2       0.56      0.62      0.59        47

    accuracy                           0.81      6545
   macro avg       0.72      0.63      0.63      6545
weighted avg       0.80      0.81      0.77      6545

Category: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5411
           1       0.88      0.40      0.55      1134

    accuracy                           0.89      6545
   macro avg       0.88      0.70      0.74      6545
weighted avg       0.89      0.89      0.87      6545

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6513
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6545
   macro avg       0.5

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6360
           1       0.67      0.03      0.06       185

    accuracy                           0.97      6545
   macro avg       0.82      0.52      0.52      6545
weighted avg       0.96      0.97      0.96      6545

Category: security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6412
           1       0.00      0.00      0.00       133

    accuracy                           0.98      6545
   macro avg       0.49      0.50      0.49      6545
weighted avg       0.96      0.98      0.97      6545

Category: military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6311
           1       0.75      0.05      0.10       234

    accuracy                           0.97      6545
   macro avg       0.86      0.53      0.54      6545
weighted avg       0.96      0.97   

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5699
           1       0.75      0.02      0.03       846

    accuracy                           0.87      6545
   macro avg       0.81      0.51      0.48      6545
weighted avg       0.86      0.87      0.82      6545

Category: infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6077
           1       0.00      0.00      0.00       468

    accuracy                           0.93      6545
   macro avg       0.46      0.50      0.48      6545
weighted avg       0.86      0.93      0.89      6545

Category: transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6241
           1       0.84      0.07      0.13       304

    accuracy                           0.96      6545
   macro avg       0.90      0.53      0.55      6545
weighted avg       0.

/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/DataScience/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6044
           1       0.92      0.39      0.54       501

    accuracy                           0.95      6545
   macro avg       0.93      0.69      0.76      6545
weighted avg       0.95      0.95      0.94      6545

Category: storm
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5942
           1       0.82      0.48      0.61       603

    accuracy                           0.94      6545
   macro avg       0.88      0.74      0.79      6545
weighted avg       0.94      0.94      0.94      6545

Category: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6476
           1       0.00      0.00      0.00        69

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98

### 8. Improving your model further.
* Leveraging  another machine learning algorithms like Logistic Regression

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.